In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from yadbil.data.processing import calculate_idf
from yadbil.graph import GraphProcessor, find_similar_posts_pagerank, get_graph_plot

In [2]:
INPUT_DATA = "data/result.json"

GRAPH_FILE_PATH = "data/filtered_graph.graphml"
POSTS_FILE_PATH = "data/posts.json"
POSTS_VIEW_FILE_PATH = "data/posts_view.json"

Data processing

In [ ]:
import json
from pathlib import Path

from yadbil.data.text.processing import TextPreprocessor


text_processor = TextPreprocessor()


def get_stemmed(text):
    return list(text_processor.preprocess(text)["stemmed_to_words"])

In [ ]:
data = []
for p in Path("data/tg_data/clean").glob("*"):
    with open(p) as file:
        d = [{"channel": p.stem, **json.loads(x)} for x in file]
        d = [{**x, "stemmed_words": get_stemmed(x["text_no_links"])} for x in d]
    data.extend(d)

with open("data/tg_data/whole_stemmed.json", "w") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [4]:
with open("data/tg_data/whole_stemmed.json") as file:
    data = json.load(file)

In [ ]:
import bm25s


# Create the BM25 model and index the corpus
retriever = bm25s.BM25()
retriever.index([x["stemmed_words"] for x in data])

In [ ]:
# Query the corpus
query = "мерж моделей"
query_tokens = get_stemmed(query)

# Get top-k results as a tuple of (doc ids, scores). Both are arrays of shape (n_queries, k)
results, scores = retriever.retrieve([query_tokens], k=10)

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.2f}): {doc}")
    print("-")
    print(data[doc]["channel"])
    print("-")
    print(data[doc]["orig_text"])

    print("-" * 50)


# ...and load them when you need them
# import bm25s
# reloaded_retriever = bm25s.BM25.load("animal_index_bm25", load_corpus=True)
# set load_corpus=False if you don't need the corpus

In [7]:
# You can save the arrays to a directory...
retriever.save("data/bm_25_index_tg")

Graph preparation

In [ ]:
idf_scores = calculate_idf(data, min_max_scale=True)

graph_proc = GraphProcessor(data, idf_scores)

print(f"Number of nodes: {graph_proc.G.number_of_nodes()}")
print(f"Number of edges: {graph_proc.G.number_of_edges()}")

Filter graph

In [ ]:
print("Full graph:", len(graph_proc.G.edges))
graph_proc.scale_edge_weights()
G_filtered = graph_proc.filter_edges_by_threshold(threshold=0.4)
print("Filtered graph:", len(G_filtered.edges))

Testing

Visualization

In [ ]:
fig = get_graph_plot(G_filtered, idf_scores)
fig.show()

In [ ]:
# Example usage
post_id = 2759  # Replace with the desired post ID
top_n = 5  # Replace with the desired number of top similar posts

similar_posts = find_similar_posts_pagerank(G_filtered, post_id, top_n)

print(f"Top {top_n} similar posts to post {post_id}:")
for post, score in similar_posts:
    print(f"Post ID: {post}, Similarity Score: {score:.4f}")

In [ ]:
# Example: Accessing edge weights
for u, v, data in G_filtered.edges(552, data=True):
    weight = data.get("weight", 0)  # Defaulting to 0 if weight not present
    print(f"Edge from {u} to {v} with weight: {weight}")

In [ ]:
# TODO: improve comparison

# a = set(posts_view["channel1150855655"][545]['stemmed_words'])
# b = set(posts_view["channel1150855655"][905]['stemmed_words'])

# print(posts_view["channel1150855655"][700]['text'])

# a.intersection(b)
# G_filtered.nodes[545]

Save

In [ ]:
import json

import networkx as nx


G_converted = G_filtered.copy()

for node, attrs in G_converted.nodes(data=True):
    for attr_key, attr_value in attrs.items():
        attrs[attr_key] = json.dumps(attr_value)

nx.write_graphml(G_converted, GRAPH_FILE_PATH)

In [ ]:
# with open(POSTS_FILE_PATH, "w") as file:
#     json.dump(posts, file, indent=4, ensure_ascii=False)

# with open(POSTS_VIEW_FILE_PATH, "w") as file:
#     json.dump(posts_view, file, indent=4, ensure_ascii=False)

Check loading

NOTE: keys from dict from int transform into str after loading

In [ ]:
from yadbil.graph.io import load_resources


GRAPH_FILE_PATH = "data/filtered_graph.graphml"
POSTS_FILE_PATH = "data/posts.json"
POSTS_VIEW_FILE_PATH = "data/posts_view.json"

G_filtered_2, posts_2, posts_view_2 = load_resources(GRAPH_FILE_PATH, POSTS_FILE_PATH, POSTS_VIEW_FILE_PATH)

In [ ]:
# G_filtered_2 == G_filtered
# len(G_filtered_2), len(G_filtered)
# len(G_filtered_2.edges), len(G_filtered.edges)
# list(posts_view.values())[0][65]
# list(posts_view_2.values())[0]["65"]

In [ ]:
# Example usage
post_id = 545  # Replace with the desired post ID
top_n = 10  # Replace with the desired number of top similar posts

similar_posts = find_similar_posts_pagerank(G_filtered_2, str(post_id), top_n)

print(f"Top {top_n} similar posts to post {post_id}:")
for post, score in similar_posts:
    print(f"Post ID: {post}, Similarity Score: {score:.4f}")